# Conversion of HDF5 to GeoTIFF

In order to work with Night Light images, you need to convert them first into GeoTIFF files. Usually you download the products in HDF5 format from the NASA website.      

This code only works for HDF5 files!    
For HDF4 files please consult https://blackmarble.gsfc.nasa.gov/Tools.html

In [1]:
from osgeo import gdal
import os
from pprint import pprint
import re

gdal.UseExceptions() # this allows translating gdal errors into python exceptions
# An exception is a Python object that represents an error

In [2]:
# function to create a list of only hdf files
def onlyh5 (files):
    newFiles = []
    for item in files:
        x = re.search("\.h5$", item)
        if x != None:
            newFiles.append(item)
    return newFiles

In [7]:
# variables
fileExtension = ".tif"
EPSG = "-a_srs EPSG:4326" #WGS84

# specify folder with hdf files and output folder
filepath = "U:/Ulloa/advanced_remote_sensing_2022/data/HDF"
outputFolder = "U:/Ulloa/advanced_remote_sensing_2022/data/results/"

# change working directory to filepath
os.chdir(filepath)

# create outputFolder if it doesn't exist yet
os.makedirs(outputFolder, exist_ok=True)

# get list of all hdf files in that folder
rasterFiles = os.listdir(os.getcwd())
rasterFiles = onlyh5(rasterFiles)
print(len(rasterFiles), "HDF5 Files found")

2 HDF5 Files found


In [8]:
## Look at the subdatasets of your hdf file
# you can also look at the documentation of your product to see this information
hdffile = gdal.Open(rasterFiles[0], gdal.GA_ReadOnly)
pprint(hdffile.GetSubDatasets())

[('HDF5:"VNP46A2.A2020214.h10v04.001.2021076005435.h5"://HDFEOS/GRIDS/VNP_Grid_DNB/Data_Fields/DNB_BRDF-Corrected_NTL',
  '[2400x2400] //HDFEOS/GRIDS/VNP_Grid_DNB/Data_Fields/DNB_BRDF-Corrected_NTL '
  '(16-bit unsigned integer)'),
 ('HDF5:"VNP46A2.A2020214.h10v04.001.2021076005435.h5"://HDFEOS/GRIDS/VNP_Grid_DNB/Data_Fields/DNB_Lunar_Irradiance',
  '[2400x2400] //HDFEOS/GRIDS/VNP_Grid_DNB/Data_Fields/DNB_Lunar_Irradiance '
  '(16-bit unsigned integer)'),
 ('HDF5:"VNP46A2.A2020214.h10v04.001.2021076005435.h5"://HDFEOS/GRIDS/VNP_Grid_DNB/Data_Fields/Gap_Filled_DNB_BRDF-Corrected_NTL',
  '[2400x2400] '
  '//HDFEOS/GRIDS/VNP_Grid_DNB/Data_Fields/Gap_Filled_DNB_BRDF-Corrected_NTL '
  '(16-bit unsigned integer)'),
 ('HDF5:"VNP46A2.A2020214.h10v04.001.2021076005435.h5"://HDFEOS/GRIDS/VNP_Grid_DNB/Data_Fields/Latest_High_Quality_Retrieval',
  '[2400x2400] '
  '//HDFEOS/GRIDS/VNP_Grid_DNB/Data_Fields/Latest_High_Quality_Retrieval '
  '(8-bit unsigned character)'),
 ('HDF5:"VNP46A2.A2020214.h10

In [6]:
## Save chosen subdatasets as tif
# for every HDF5 file in the array
for item in rasterFiles:

    # to work with .hdf5 and .h5 files
    rasterFilePre = re.sub("\.hdf5|\.h5", "", item)
    
    ## Open HDF file
    hdflayer = gdal.Open(item, gdal.GA_ReadOnly)
    
    ## Open raster layer
    # hdflayer.GetSubDatasets()[0][0] - for first layer (ntl)
    # hdflayer.GetSubDatasets()[2][0] - for third layer (gap-filled ntl) ...etc
    # [hdflayer.GetSubDatasets()[0][0], hdflayer.GetSubDatasets()[2][0]] - for multiple layers
    ""
    subhdflayer = [hdflayer.GetSubDatasets()[0][0]]
    ""
    # for every layer create new tif file
    for subItem in subhdflayer:
        rlayer = gdal.Open(subItem, gdal.GA_ReadOnly)
        outputName = re.search("Data_Fields/.+$", subItem)
        outputName = outputName.string[(outputName.start() + 12):]
        outputNameNoSpace = outputName.strip().replace(" ","_").replace("/","_")
        outputNameFinal = rasterFilePre + "_" + outputNameNoSpace + fileExtension
        outputRaster = outputFolder + outputNameFinal
        
        #collect bounding box coordinates
        HorizontalTileNumber = int(rlayer.GetMetadata_Dict()["HorizontalTileNumber"])
        VerticalTileNumber = int(rlayer.GetMetadata_Dict()["VerticalTileNumber"])
        
        WestBoundCoord = (10*HorizontalTileNumber) - 180
        NorthBoundCoord = 90-(10*VerticalTileNumber)
        EastBoundCoord = WestBoundCoord + 10
        SouthBoundCoord = NorthBoundCoord - 10
        
        translateOptionText = EPSG + " -a_nodata 65535 " + " -a_ullr " + str(WestBoundCoord) + " " + str(NorthBoundCoord) + " " + str(EastBoundCoord) + " " + str(SouthBoundCoord)
        translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine(translateOptionText))
        gdal.Translate(outputRaster, rlayer, options=translateoptions)
        
        #Display image in QGIS (run it within QGIS python Console) - remove comment to display
        #iface.addRasterLayer(outputRaster, outputNameFinal)

NameError: name 'iface' is not defined